# Some More Information on using Tequila for Quantum Chemistry #
## Measurement Schemes ##

Here we show how to use methods in Tequila for partitioning Hamiltonians into compatible sets.
We will cover:
* Initialization of molecules within tequila (briefly)
* Usage of different qubit encodings (JW, BK....)(briefly)
* Using a Qubit-wise commutativity grouping scheme
* Using a Full-commutativity grouping scheme
* Generating measurement circuits

Papers mentioned by order of appearance:
1. [The Bravyi-Kitaev transformation for quantum computation of electronic structure](https://arxiv.org/pdf/1208.5986)
2. [Minimizing State Preparations in Variational Quantum Eigensolver by Partitioning into Commuting Families](https://arxiv.org/abs/1907.13623v1)
3. [Cartan sub-algebra approach to efficient measurements of quantum observables](https://arxiv.org/abs/2007.01234)
4. [Deterministic improvements of quantum measurements with grouping of compatible
operators, non-local transformations, and covariance estimates](https://arxiv.org/abs/2201.01471)
5. [$O(N^3)$ Measurement Cost for Variational Quantum Eigensolver on Molecular Hamiltonians](https://arxiv.org/abs/1908.11857)
6. [Fidelity overhead for non-local measurements in variational quantum algorithms](https://arxiv.org/abs/2205.07113)

This tutorial is to showcase the different methods available to partition qubit Hamiltonians into sets of compatible (commuting) operators and calculate expectation values for molecular electronic Hamiltonians. Here we will make use of the transformations available to map Hamiltonians to an iso-spectral qubit Hamiltonian with some fermionic -> qubit operator transform such as the Bravyi-Kitaev transformation or the Jordan-Wigner transformation. If you haven't yet already, see the tutorial [here](https://github.com/tequilahub/tequila-tutorials/blob/main/chemistry/ChemistryModule.ipynb) about basic chemistry usage with Tequila and the qubit transforms available (and for some extra reading material on qubit transforms this [paper](https://arxiv.org/pdf/1208.5986) gives a great introduction). We will be using Tequila's `Molecule` function for instantiating molecules from data on inter-nuclear distance and geometry also known as `.xyz` files. This requires `Psi4` to be installed so if you haven't already yet be sure to run `conda install psi4 -c psi4`.  
 
Let's get started with a simple(ish) molecule LiH and get a qubit Hamiltonian to measure using the Bravyi-Kitaev transformation.

In [56]:
import tequila as tq 
#Instantiating our molecule from a geometry string with Molecule
molecule = tq.chemistry.Molecule(geometry = "H 0.0 0.0 0.0\nLi 0.0 0.0 1.6", basis_set="sto-3g", transformation="BravyiKitaev")
H =  molecule.make_hamiltonian()

To begin with, why do we even need to bother with this extra work of grouping? Can we not just measure the qubit Hamiltonian directly? The answer I'm afraid is 'no'. Current quantum hardware is limited to projective measurements onto the $\hat{Z}$ computational basis. If we print out our qubit Hamiltonian we will see that we have a qubit Hamiltonian that is a sum of *many* (emphasis on many) Pauli words. Pauli Words/Pauli Products are tensor products of the Pauli spin matrices, which contain $\hat{X}$,$\hat{Y}$, and $\hat{Z}$ terms. So this is no bueno.
<!-- 
However, this task itself is not as straightforward as it seems either. Finding unitaries that will rotate $\hat{H}_q$ into an all-Z form (Ising form) can be a challenging task, especially for Hamiltonians that have a large number of terms. -->

In [57]:
print("The qubit Hamiltonian has {} terms".format(len(H)),'\n')
print('H =',H)

The qubit Hamiltonian has 276 terms 

H = -5.7342-0.2985Z(0)-0.0036X(0)Y(1)Y(2)+0.0036Y(0)Y(1)X(2)-0.0132X(0)X(1)X(3)Y(7)Y(8)X(9)+0.0132Y(0)X(1)X(3)Y(7)X(8)X(9)-0.2985Z(0)Z(1)+0.0036Z(0)X(1)Z(3)-0.0036X(1)Z(2)-0.0132Z(0)X(1)X(3)Y(7)Y(9)+0.0132Y(1)X(3)Y(7)Z(8)X(9)-0.3905Z(2)+0.0241Z(1)X(2)X(3)Y(7)Y(8)X(9)-0.0241Z(1)Y(2)X(3)Y(7)X(8)X(9)-0.3905Z(1)Z(2)Z(3)+0.0241Z(1)Z(2)X(3)Y(7)Y(9)-0.0241Y(3)Y(7)Z(8)X(9)-0.4262Z(4)-0.4262Z(4)Z(5)-0.4262Z(6)-0.4262Z(3)Z(5)Z(6)Z(7)-0.5637Z(8)-0.5637Z(8)Z(9)+0.1218Z(1)+0.0121Y(0)X(1)Y(2)+0.0121X(0)X(1)X(2)+0.0317Y(0)Y(1)X(3)Y(7)Y(8)X(9)+0.0317X(0)Y(1)X(3)Y(7)X(8)X(9)-0.0121X(1)Z(3)+0.0121Z(0)X(1)Z(2)+0.0033Y(0)Z(1)Y(2)Z(3)+0.0033X(0)Z(1)X(2)+0.0033X(0)Z(1)X(2)Z(3)+0.0033Y(0)Z(1)Y(2)-0.0087Y(0)Y(3)Y(7)Y(8)X(9)-0.0087X(0)Z(2)X(3)Y(7)Y(8)X(9)-0.0087X(0)Y(3)Y(7)X(8)X(9)+0.0087Y(0)Z(2)X(3)Y(7)X(8)X(9)+0.0059Y(0)Y(4)Z(5)+0.0059X(0)Z(1)X(4)+0.0059X(0)X(4)Z(5)+0.0059Y(0)Z(1)Y(4)+0.0059Y(0)Z(3)Z(5)Y(6)Z(7)+0.0059X(0)Z(1)X(6)+0.0059X(0)Z(3)Z(5)X(6)Z(7)+0.0059Y(0)Z(1)

This means that before we do any measurements of $\hat{H}_q$ we need additional unitaries that will rotate Pauli words $\hat{P}_n$ to an all-$\hat{Z}$ form (Ising form). In other words, for a qubit Hamiltonian $\hat{H}_q = \sum_n a \hat{P}_n$, we want a set of unitaries $\hat{U}_n$ such that $$\hat{Z}_n = \hat{U}_n \hat{P}_n \hat{U}^{\dagger}_n$$ "Great, let's go ahead do that!", you may say, being the keen quantum computing enthusiast you are. I like your enthusiasm. However, there is another reason we printed out $\hat{H}_q$ in the cell above. Look at the number of terms we have up there in $\hat{H}_q$ (276!). That is a lot of terms to measure. And the more orbitals we use in our basis set, the more terms we are going to have to measure. In general the number of terms in a qubit Hamiltonian will scale in Big-O notation as $O(N^4)$, with $N$ being the number of spin orbitals used in our basis set.

So to make life a lot easier for ourselves a trick we can use is grouping commuting Pauli words into sets where they all are compatible with each other, meaning they can all be put into Ising form by the same unitary and thus be measured simultaneously! 🤩

So we want to partition the qubit Hamiltonian such that $\hat{H}_q = \sum \hat{H}_{\alpha}$, where the $\hat{H}_{\alpha}$ are our commuting sets. The expectation value for $\hat{H}_q$ is then: $$ \bra{\Psi} \hat{H}_q \ket{\Psi} = \bra{\Psi} \sum_\alpha \hat{H}_\alpha \ket{\Psi} = \sum_{\alpha} \bra{\Psi} \hat{U}_{\alpha}^{\dagger} \hat{U}_{\alpha} \hat{H}_\alpha \hat{U}^{\dagger}_{\alpha} \hat{U}_{\alpha} \ket{\Psi}  = \sum_{\alpha} \bra{\Phi_\alpha} \hat{U}_{\alpha} \hat{H}_\alpha \hat{U}^{\dagger}_{\alpha} \ket{\Phi_\alpha} =  \sum_{\alpha} \bra{\Phi_{\alpha}}  \hat{Z}_{\alpha} \ket{\Phi_{\alpha}} $$ For qubit hamiltonians there are two main grouping methods that we can employ. The first is to find sets where they are all commuting in a qubit-wise commuting (QWC) fashion, meaning that each Pauli word commutes with each other, and each Pauli operator with the same index within words commutes with each other also. The other less restrictive method is to group them where they all commute with each other in a normal fashion $[\hat{P}_i,\hat{P}_j] = 0$, also known as fully commuting (FC). However, there are trade-offs to consider. QWC grouping is nice because the circuits used for measuring will only contain rotations localized to individual qubits and will also thus have shallow depths. However, the trade-off is that grouping this way doesn't reduce the number of terms to measure by much. It has been [empirically shown](https://arxiv.org/abs/1907.13623v1) that this grouping method will only reduce the prefactor in the $O(N^4)$ scaling by three. 

Before we get into the FC grouping methods, let's see how we can use functions found in Tequila's chemistry modules to develop measurement circuits with a QWC grouping scheme.

We begin with calling the function `compile_commuting_parts` found in `compile_groups.py`, and pass to it the keyword argument that specifies the grouping method we want to use. See [this paper here](https://journals.aps.org/prxquantum/pdf/10.1103/PRXQuantum.2.040320) comparing different grouping methods in fermionic and qubit algebras for some more in-depth reading. It's wise to choose a grouping method that performs efficiently in the number of measurements needed for each Hamiltonian. As discussed in the paper, for qubit Hamiltonians this is (typically) best achieved using the Greedy Grouping method (Sorted Insertion). This heuristic groups Pauli words by adding as many Pauli words with large magnitude coefficients together as possible. Doing this tends to minimize the variances, and thus the number of measurements needed. 

So using this function we will pass the options dictionary with the keyword for the method being `'si'` (for using sorted insertion grouping method), and the keyword for the commutativity condition being `'qwc'` (for qubit-wise commutativity). If you wish to use qwc grouping, it is best advised to turn off the circuit-optimizing procedure, since that only performs well for circuits that require multi-qubit (entangling) gates. So we also specify `optimizing = 'no'`.


In [58]:
from tequila.grouping.compile_groups import compile_commuting_parts 
optimizing = 'no'
options = {
    'method': 'si',  # or any other method like 'lf', 'rlf', 'ics'
    'condition': 'qwc',  # fully commuting condition
    # You can add other options here as needed
}
qwc_groups_and_unitaries, sample_ratios = compile_commuting_parts(H, unitary_circuit=optimizing, options=options)
number_qwc_groups = len(qwc_groups_and_unitaries)

print('The number of groups to measure is: ', number_qwc_groups)
print('We have reduced the number of terms to measure to:',number_qwc_groups/len(H) )

The number of groups to measure is:  69
We have reduced the number of terms to measure to: 0.25


This function has returned to `qwc_groups_and_unitaries` a list of tuples containing the qubit-wise commuting partitions rotated to all z form, and the circuit (U_n) that will do the proper rotations (`tuple(all-z partition, measurement circuit)`). We can ignore the `sample_ratios` returned here since it will be an empty list. This list is important when we want to use other grouping methods that split coefficients, for putting Pauli products in more than one commuting group (see: the [Iterative Coefficient Splitting method](https://arxiv.org/pdf/2201.01471))

So for example, if we want to measure the second clique (the first  clique is already all z), the term we need to measure and the circuit that will do the proper rotations on our quantum state are:

In [59]:
from tequila import QCircuit
group_1 = qwc_groups_and_unitaries[1][0]
u_1 = qwc_groups_and_unitaries[1][1]
print('The group we need to measure: ', group_1)

tq.draw(u_1,backend='qiskit')

The group we need to measure:  -0.0336Z(0)Z(1)Z(3)Z(7)Z(8)Z(9)-0.0317Z(1)Z(3)Z(7)Z(9)-0.0132Z(0)Z(1)Z(3)Z(7)Z(9)+0.0121Z(0)Z(1)Z(2)+0.0108Z(1)Z(2)Z(8)+0.0042Z(4)Z(5)Z(6)+0.0042Z(0)Z(2)Z(3)Z(7)Z(9)-0.0036Z(1)Z(2)+0.0031Z(0)Z(1)Z(2)Z(3)Z(7)Z(9)-0.0019Z(1)
circuit: 
Rz(target=(1,), parameter=-1.5707963267948966)
Rx(target=(1,), parameter=-1.5707963267948966)
Rz(target=(1,), parameter=-1.5707963267948966)
Rz(target=(3,), parameter=-1.5707963267948966)
Rx(target=(3,), parameter=-1.5707963267948966)
Rz(target=(3,), parameter=-1.5707963267948966)
Rz(target=(7,), parameter=-1.5707963267948966)
Ry(target=(7,), parameter=-1.5707963267948966)
Rz(target=(7,), parameter=-1.5707963267948966)
Rz(target=(9,), parameter=-1.5707963267948966)
Ry(target=(9,), parameter=-1.5707963267948966)
Rz(target=(9,), parameter=-1.5707963267948966)
Rz(target=(4,), parameter=-1.5707963267948966)
Rx(target=(4,), parameter=-1.5707963267948966)
Rz(target=(4,), parameter=-1.5707963267948966)
Rz(target=(6,), parameter=-1.57

On the other hand, Tequila also has the option to use a full commutativity grouping scheme for quit operators. The main advantage the FC grouping scheme presents is that it [has been shown](https://arxiv.org/pdf/1908.11857v1) to result in a reduction in the number of terms to measure from $O(N^4) \rightarrow O(N^3)!$ The tradeoff however is the need for entangling gates in measurement circuits, which can be a concern considering the fidelity of multi-qubit operators with the hardware being used. Recent results [have shown](https://arxiv.org/abs/2205.07113) however that using FC grouping in tandem with a circuit optimization procedure to reduce CNOT gate counts can still result in needing fewer overall measurements than the QWC grouping scheme. And this is already in Tequila so no need to code this up yourself. 🥳

We can use the following code to implement an FC grouping scheme, with a measurement circuit optimization protocol for the LiH qubit Hamiltonian. Again we will pass `'si'` to use the Greedy Grouping method, and this time specify `'fc'` for full commutativity. To use the circuit optimization procedure ([see this paper here for the details](https://arxiv.org/abs/2205.07113)), we pass the argument `'improved'` to `unitary_circuit`.

In [60]:
options = {
    'method': 'si',  # or any other method like 'lf', 'rlf', 'ics'
    'condition': 'fc',  # fully commuting condition
    # You can add other options here as needed
}

fc_groups_and_unitaries, sample_ratios = compile_commuting_parts(H, unitary_circuit='improved', options=options)
number_fc_groups = len(fc_groups_and_unitaries)
print('The number of groups to measure is: ', number_fc_groups)
print('We have reduced the number of terms to measure by:',number_fc_groups/len(H) )

The number of groups to measure is:  21
We have reduced the number of terms to measure by: 0.07608695652173914
